In [2]:
import pandas as pd
user_data = pd.read_table('data/user_db.csv',sep=',')
user_data

,user_id,gender,area,age,access_media,group,discounted
0,78ff837c7cea9f33,남,송파구,20,인스타그램,1,10000.0
1,2987aa87d23c9e3d,남,강서구,30,검색,1,30000.0
2,23182525e9996864,남,도봉구,20,인스타그램,1,20000.0
3,ffb6913842d617a5,여,용산구,20,인스타그램,1,20000.0
4,2c6822e7e833dc95,여,은평구,30,네이버,1,30000.0
...,...,...,...,...,...,...,...
8123,6da398d852411234,여,은평구,30,지인추천,0,NaN
8124,6da398d852411235,남,강서구,30,검색,0,NaN
8125,6da398d852411236,여,용산구,20,인스타그램,0,NaN
8126,6da398d852411237,남,광진구,30,네이버,0,NaN


In [3]:
#각 할인별 신청한 수강생 수 구하기
students_discounted = user_data.groupby('discounted')['user_id'].count()
students_discounted

discounted
10000.0     808
20000.0    1588
30000.0    3258
Name: user_id, dtype: int64

In [34]:
cohort_data = pd.read_table('data/cohort_data.csv',sep=',')

#날짜 데이터 타입 변경
cohort_data['start_time'] = pd.to_datetime(cohort_data['created_at'], format='mixed')

#시작 week 구하기
cohort_data['start_week']= cohort_data['start_time'].dt.isocalendar().week

#시작 주 범위 알기
category_range = set(cohort_data['start_week'])

# 범주화 하기
progress_rate = list(cohort_data['progress_rate'])

#범주를 구분하는 기준 bins
bins = [0,4.11,26.03,41.10,61.64,80.82,100]

#구분한 범주의 라벨 labels
labes=[0,1,2,3,4,5]

#범주화에 사용하는 함수 pd.cut
cuts = pd.cut(progress_rate,bins, right=True,include_lowest=True, labels=labes)
cuts = pd.DataFrame(cuts)

# 표 합치기
cohort_data = pd.concat([cohort_data,cuts],axis=1, join='inner')

#표 인덱스 변경하기
cohort_data.columns=['created_at','user_id','name','progress_rate','start_time','start_week',"week"]

#시작주와, 수강 주차별 기준으로 표 grouping 하기
grouping = cohort_data.groupby(['start_week','week'])

#시작주와, 수강 주차별에 해당하는 수강생 수 구하기
cohort_data = grouping['user_id'].apply(pd.Series.nunique)
cohort_data = pd.DataFrame(cohort_data)

#각 주차별 수강한 수강생 총 합 구하기
k=31
for i in range(6):
  for j in range(5, 0, -1):
    cohort_data.at[(k,j-1), 'user_id'] = int(cohort_data.at[(k,j),'user_id']) +  int(cohort_data.at[(k,j-1),'user_id'])
  k=k+1
cohort_data = cohort_data.reset_index()
cohort_data.head()


cohort_counts = cohort_data.pivot(index="start_week",
                                  columns="week",
                                  values="user_id")

# 앞서 만든 피벗 테이블을 retention 변수에 저장하기
retention = cohort_counts
#각 주(week) 별 최초 수강생 수만 가져오기
cohort_sizes = cohort_counts.iloc[:,0]


# 최초 수강생 수를 각 데이터에 나눠주기
retention = cohort_counts.divide(cohort_sizes, axis=0)

#각 수치 퍼센트로 변경하기
retention.round(3)*100

/var/folders/gb/jqxp5nqd3dz14l_g216xwzdm0000gn/T/ipykernel_98054/4172012388.py:32: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouping = cohort_data.groupby(['start_week','week'])


week,0,1,2,3,4,5
start_week,,,,,,
31,100.0,97.8,91.7,85.7,78.7,72.6
32,100.0,98.8,93.9,79.9,67.7,61.9
33,100.0,95.8,83.7,78.9,69.9,63.7
34,100.0,94.6,88.9,82.8,77.8,64.8
35,100.0,92.9,84.9,80.0,72.0,60.9
36,100.0,96.9,89.9,84.0,76.0,65.9


In [35]:
# 1) 주차 간 전환율 리텐션 테이블 만들기
#첫 시작주가 31주이죠!
w=31
#36주까지 있으니, for 구문의 range를 6으로 둡시다
for i in range(6):
	#각 시작 주차별 수강 주차는 총 0부터 5까지의 범위가 있죠? 그래서 range(1, 6, 1)로 설정합니다. 1부터 시작하여, 5까지 1씩 커진다는 뜻 입니다!
    for j in range(5, 1, -1):
        retention.at[(w,j)] = retention.at[(w,j)] / retention.at[(w,j-1)]
    w = w+1

retention

week,0,1,2,3,4,5
start_week,,,,,,
31,1.0,0.978261,0.937778,0.933649,0.918782,0.922652
32,1.0,0.988372,0.950000,0.851393,0.847273,0.914163
33,1.0,0.958478,0.873646,0.942149,0.885965,0.910891
34,1.0,0.946360,0.939271,0.931034,0.939815,0.832512
35,1.0,0.929231,0.913907,0.942029,0.900000,0.846154
36,1.0,0.968641,0.928058,0.934109,0.904564,0.866972


In [36]:
cohort_counts

week,0,1,2,3,4,5
start_week,,,,,,
31,230,225,211,197,181,167
32,344,340,323,275,233,213
33,289,277,242,228,202,184
34,261,247,232,216,203,169
35,325,302,276,260,234,198
36,287,278,258,241,218,189


In [ ]:
retention_ = cohort_counts.copy()
